# Sentiment Classification From Text & How To "Frame Problems" for a Neural Network

Material basado(con modificaciones personalizadas) en: 
Sentiment Classification & How To "Frame Problems for a Neural Network" de Udacity
* Parte de "deep learning nanodegree" de udacity.
* Instructor: Andrew Trask

### What You Should Already Know

- neural networks, forward and back-propagation
- stochastic gradient descent
- loss functions: mean squared error, cross-entropy
- and train/test splits

### Tutorial Outline:

- Intro: The Importance of "Framing a Problem" (this lesson)

- [Curate a Dataset](#lesson_1)
- [Developing a "Predictive Theory"](#lesson_2)
- [**PROJECT 1**: Quick Theory Validation](#project_1)


- [Transforming Text to Numbers](#lesson_3)
- [**PROJECT 2**: Creating the Input/Output Data](#project_2)


- Putting it all together in a Neural Network (video only - nothing in notebook)
- [**PROJECT 3**: Building our Neural Network](#project_3)


- [Understanding Neural Noise](#lesson_4)
- [**PROJECT 4**: Making Learning Faster by Reducing Noise](#project_4)


- [Analyzing Inefficiencies in our Network](#lesson_5)
- [**PROJECT 5**: Making our Network Train and Run Faster](#project_5)


- [Further Noise Reduction](#lesson_6)
- [**PROJECT 6**: Reducing Noise by Strategically Reducing the Vocabulary](#project_6)


- [Analysis: What's going on in the weights?](#lesson_7)

In [ ]:
import base64
import requests

# Lesson: Curate a Dataset<a id='lesson_1'></a>


In [ ]:
reviews_url = "https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/reviews.txt"
req = requests.get(reviews_url)
reviews_raw = req.text

reviews = reviews_raw.split("\n")

In [ ]:
labels_url = "https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/labels.txt"
req = requests.get(labels_url)
labels_raw = req.text.upper()

labels = labels_raw.split("\n")

In [ ]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")


**Note:** The data in `reviews.txt` we're using has already been preprocessed a bit and contains only lower case characters. If we were working from raw data, where we didn't know it was all lower case, we would want to add a step here to convert it. That's so we treat different variations of the same word, like `The`, `the`, and `THE`, all the same way.

## Get an idea of how the data looks like

In [ ]:
len(reviews),len(labels)

(25001, 25001)

In [ ]:
reviews[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [ ]:
labels[0]

'POSITIVE'

In [ ]:
pretty_print_review_and_label(2)

POSITIVE	:	homelessness  or houselessness as george carlin stated  has been an issue for ye...


# Lesson: Develop a Predictive Theory<a id='lesson_2'></a>

In [ ]:
print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)
pretty_print_review_and_label(6267)
pretty_print_review_and_label(21934)
pretty_print_review_and_label(5297)
pretty_print_review_and_label(4998)

labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...
NEGATIVE	:	comment this movie is impossible . is terrible  very improbable  bad interpretat...
POSITIVE	:	excellent episode movie ala pulp fiction .  days   suicides . it doesnt get more...
NEGATIVE	:	if you haven  t seen this  it  s terrible . it is pure trash . i saw this about ...
POSITIVE	:	this schiffer guy is a real genius  the movie is of excellent quality and both e...


# Project 1: Quick Theory Validation<a id='project_1'></a>

You'll find the [Counter](https://docs.python.org/2/library/collections.html#collections.Counter) class to be useful in this exercise, as well as the [numpy](https://docs.scipy.org/doc/numpy/reference/) library.

In [ ]:
from collections import Counter
import numpy as np

We'll create three `Counter` objects, one for words from postive reviews, one for words from negative reviews, and one for all the words.

In [ ]:
# Create three Counter objects to store positive, negative and total counts
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

**TODO:** Examine all the reviews. For each word in a positive review, increase the count for that word in both your positive counter and the total words counter; likewise, for each word in a negative review, increase the count for that word in both your negative counter and the total words counter.

**Note:** Throughout these projects, you should use `split(' ')` to divide a piece of text (such as a review) into individual words. If you use `split()` instead, you'll get slightly different results than what the videos and solutions show.

In [ ]:
# TODO: Loop over all the words in all the reviews and increment the counts in the appropriate counter objects
for index,label in enumerate(labels):
    words = reviews[index].split(' ')
    if label == 'POSITIVE':
        for word in words:
            positive_counts[word]+=1
            total_counts[word]+=1
    else:
        for word in words:
            negative_counts[word]+=1
            total_counts[word]+=1

Run the following two cells to list the words used in positive reviews and negative reviews, respectively, ordered from most to least commonly used. 

In [ ]:
# Examine the counts of the most common words in positive reviews
positive_counts.most_common()[:100]

[('', 550468),
 ('the', 173324),
 ('.', 159654),
 ('and', 89722),
 ('a', 83688),
 ('of', 76855),
 ('to', 66746),
 ('is', 57245),
 ('in', 50215),
 ('br', 49235),
 ('it', 48025),
 ('i', 40743),
 ('that', 35630),
 ('this', 35080),
 ('s', 33815),
 ('as', 26308),
 ('with', 23247),
 ('for', 22416),
 ('was', 21917),
 ('film', 20937),
 ('but', 20822),
 ('movie', 19074),
 ('his', 17227),
 ('on', 17008),
 ('you', 16681),
 ('he', 16282),
 ('are', 14807),
 ('not', 14272),
 ('t', 13720),
 ('one', 13655),
 ('have', 12587),
 ('be', 12416),
 ('by', 11997),
 ('all', 11942),
 ('who', 11464),
 ('an', 11294),
 ('at', 11234),
 ('from', 10767),
 ('her', 10474),
 ('they', 9895),
 ('has', 9186),
 ('so', 9154),
 ('like', 9038),
 ('about', 8313),
 ('very', 8305),
 ('out', 8134),
 ('there', 8057),
 ('she', 7779),
 ('what', 7737),
 ('or', 7732),
 ('good', 7720),
 ('more', 7521),
 ('when', 7456),
 ('some', 7441),
 ('if', 7285),
 ('just', 7152),
 ('can', 7001),
 ('story', 6780),
 ('time', 6515),
 ('my', 6488),
 ('g

In [ ]:
# Examine the counts of the most common words in negative reviews
negative_counts.most_common()[:100]

[('', 561463),
 ('.', 167538),
 ('the', 163389),
 ('a', 79321),
 ('and', 74385),
 ('of', 69009),
 ('to', 68974),
 ('br', 52637),
 ('is', 50083),
 ('it', 48327),
 ('i', 46880),
 ('in', 43753),
 ('this', 40920),
 ('that', 37615),
 ('s', 31546),
 ('was', 26291),
 ('movie', 24965),
 ('for', 21927),
 ('but', 21781),
 ('with', 20878),
 ('as', 20625),
 ('t', 20361),
 ('film', 19218),
 ('you', 17549),
 ('on', 17192),
 ('not', 16354),
 ('have', 15144),
 ('are', 14623),
 ('be', 14541),
 ('he', 13856),
 ('one', 13134),
 ('they', 13011),
 ('at', 12279),
 ('his', 12147),
 ('all', 12036),
 ('so', 11463),
 ('like', 11238),
 ('there', 10775),
 ('just', 10619),
 ('by', 10549),
 ('or', 10272),
 ('an', 10266),
 ('who', 9969),
 ('from', 9731),
 ('if', 9518),
 ('about', 9061),
 ('out', 8979),
 ('what', 8422),
 ('some', 8306),
 ('no', 8143),
 ('her', 7947),
 ('even', 7687),
 ('can', 7653),
 ('has', 7604),
 ('good', 7423),
 ('bad', 7401),
 ('would', 7036),
 ('up', 6970),
 ('only', 6781),
 ('more', 6730),
 ('

As you can see, common words like "the" appear very often in both positive and negative reviews. Instead of finding the most common words in positive or negative reviews, what you really want are the words found in positive reviews more often than in negative reviews, and vice versa. To accomplish this, you'll need to calculate the **ratios** of word usage between positive and negative reviews.

**TODO:** Check all the words you've seen and calculate the ratio of postive to negative uses and store that ratio in `pos_neg_ratios`. 
>Hint: the positive-to-negative ratio for a given word can be calculated with `positive_counts[word] / float(negative_counts[word]+1)`. Notice the `+1` in the denominator – that ensures we don't divide by zero for words that are only seen in positive reviews.

In [ ]:
# Create Counter object to store positive/negative ratios
pos_neg_ratios = Counter()

#  Calculate the ratios of positive and negative uses of the most common words
#       Consider words to be "common" if they've been used at least 100 times
#  If a word is common, calculate its "positive" ratio

for word,count in total_counts.most_common():
    if count >= 100:
        positive_count = positive_counts[word]
        negative_count = negative_counts[word] +1 #(+1 to avoid division by zero)
        
        word_positive_ratio = positive_count / negative_count
        
        pos_neg_ratios[word] = word_positive_ratio

Examine the ratios you've calculated for a few words:

In [ ]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 1.0607993145235326
Pos-to-neg ratio for 'amazing' = 4.022813688212928
Pos-to-neg ratio for 'terrible' = 0.17744252873563218


Looking closely at the values you just calculated, we see the following:

* Words that you would expect to see more often in positive reviews – like "amazing" – have a ratio greater than 1. The more skewed a word is toward postive, the farther from 1 its positive-to-negative ratio  will be.
* Words that you would expect to see more often in negative reviews – like "terrible" – have positive values that are less than 1. The more skewed a word is toward negative, the closer to zero its positive-to-negative ratio will be.
* Neutral words, which don't really convey any sentiment because you would expect to see them in all sorts of reviews – like "the" – have values very close to 1. A perfectly neutral word – one that was used in exactly the same number of positive reviews as negative reviews – would be almost exactly 1. The `+1` we suggested you add to the denominator slightly biases words toward negative, but it won't matter because it will be a tiny bias and later we'll be ignoring words that are too close to neutral anyway.

Ok, the ratios tell us which words are used more often in postive or negative reviews, but the specific values we've calculated are a bit difficult to work with. A very positive word like "amazing" has a value above 4, whereas a very negative word like "terrible" has a value around 0.18. Those values aren't easy to compare for a couple of reasons:

* Right now, 1 is considered neutral, but the absolute value of the postive-to-negative rations of very postive words is larger than the absolute value of the ratios for the very negative words. So there is no way to directly compare two numbers and see if one word conveys the same magnitude of positive sentiment as another word conveys negative sentiment. So we should center all the values around netural so the absolute value fro neutral of the postive-to-negative ratio for a word would indicate how much sentiment (positive or negative) that word conveys.
* When comparing absolute values it's easier to do that around zero than one. 

To fix these issues, we'll convert all of our ratios to new values using logarithms.

**TODO:** Go through all the ratios you calculated and convert their values using the following formulas:
> * For any postive words, convert the ratio using `np.log(ratio)`
> * For any negative words, convert the ratio using `-np.log(1/(ratio + 0.01))`

That second equation may look strange, but what it's doing is dividing one by a very small number, which will produce a larger positive number. Then, it takes the `log` of that, which produces numbers similar to the ones for the postive words. Finally, we negate the values by adding that minus sign up front. In the end, extremely positive and extremely negative words will have positive-to-negative ratios with similar magnitudes but oppositite signs.

In [ ]:
# TODO: Convert ratios to logs
# Note to myself: i think for negative words a simple -1 * (ratio)**-1 would do but...

for word in pos_neg_ratios:
    ratio = pos_neg_ratios[word]
    
    if ratio > 1:
        new_ratio  = np.log(ratio)
    else:
        new_ratio = -np.log(1/(ratio +0.001))
    
    pos_neg_ratios[word] = new_ratio

Examine the new ratios you've calculated for the same words from before:

In [ ]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 0.05902269426102881
Pos-to-neg ratio for 'amazing' = 1.3919815802404802
Pos-to-neg ratio for 'terrible' = -1.723488697472832


If everything worked, now you should see neutral words with values close to zero. In this case, "the" is near zero but slightly positive, so it was probably used in more positive reviews than negative reviews. But look at "amazing"'s ratio - it's above `1`, showing it is clearly a word with positive sentiment. And "terrible" has a similar score, but in the opposite direction, so it's below `-1`. It's now clear that both of these words are associated with specific, opposing sentiments.

Now run the following cells to see more ratios. 

The first cell displays all the words, ordered by how associated they are with postive reviews. (Your notebook will most likely truncate the output so you won't actually see *all* the words in the list.)

The second cell displays the 30 words most associated with negative reviews by reversing the order of the first list and then looking at the first 30 words. (If you want the second cell to display all the words, ordered by how associated they are with negative reviews, you could just write `reversed(pos_neg_ratios.most_common())`.)

You should continue to see values similar to the earlier ones we checked – neutral words will be close to `0`, words will get more positive as their ratios approach and go above `1`, and words will get more negative as their ratios approach and go below `-1`. That's why we decided to use the logs instead of the raw ratios.

In [ ]:
# words most frequently seen in a review with a "POSITIVE" label
pos_neg_ratios.most_common()[:30]

[('edie', 4.6913478822291435),
 ('paulie', 4.07753744390572),
 ('felix', 3.152736022363656),
 ('polanski', 2.8233610476132043),
 ('matthau', 2.80672172860924),
 ('victoria', 2.681021528714291),
 ('mildred', 2.6026896854443837),
 ('gandhi', 2.538973871058276),
 ('flawless', 2.451005098112319),
 ('superbly', 2.26002547857525),
 ('perfection', 2.159484249353372),
 ('astaire', 2.1400661634962708),
 ('captures', 2.038619547159581),
 ('voight', 2.030170492673053),
 ('wonderfully', 2.0218960560332353),
 ('powell', 1.978345424808467),
 ('brosnan', 1.9547990964725592),
 ('lily', 1.9203768470501485),
 ('bakshi', 1.9029851043382795),
 ('lincoln', 1.9014583864844796),
 ('refreshing', 1.8551812956655511),
 ('breathtaking', 1.8481124057791867),
 ('bourne', 1.8478489358790986),
 ('lemmon', 1.8458266904983307),
 ('delightful', 1.8002701588959635),
 ('flynn', 1.7996646487351682),
 ('andrews', 1.7764919970972666),
 ('homer', 1.7692866133759964),
 ('beautifully', 1.7626953362841438),
 ('soccer', 1.757857

In [ ]:
# words most frequently seen in a review with a "NEGATIVE" label
list(reversed(pos_neg_ratios.most_common()))[0:30]

# Note: 
#       If you explore the documentation for the Counter class, 
#       you will see you could also find the 30 least common
#       words like this: pos_neg_ratios.most_common()[:-31:-1]

[('boll', -4.835282406618394),
 ('uwe', -4.527846102553548),
 ('seagal', -3.606606956743819),
 ('unwatchable', -3.232428791272904),
 ('stinker', -3.1843768086123894),
 ('mst', -2.931339111817795),
 ('incoherent', -2.918210222585477),
 ('unfunny', -2.677582826320177),
 ('waste', -2.6057506568316997),
 ('blah', -2.5574420885289504),
 ('horrid', -2.4729780789227265),
 ('pointless', -2.443723518768643),
 ('atrocious', -2.4146593168481547),
 ('redeeming', -2.3576171050548607),
 ('prom', -2.35030978724235),
 ('drivel', -2.33663658094709),
 ('lousy', -2.2975727999267512),
 ('worst', -2.2771908066767406),
 ('laughable', -2.2547849053935245),
 ('awful', -2.2179631752505466),
 ('poorly', -2.2115829815780015),
 ('wasting', -2.1955788734265296),
 ('remotely', -2.1882648359647474),
 ('existent', -2.0714733720306593),
 ('boredom', -1.987774353154012),
 ('miserably', -1.9851235895077681),
 ('sucks', -1.9798005773223373),
 ('uninspired', -1.9760572894714397),
 ('lame', -1.9745380983043672),
 ('insult'

# End of Project 1. 


# Transforming Text into Numbers<a id='lesson_3'></a>


In [ ]:
from IPython.display import Image

review = "This was a horrible, terrible movie."

#Image(filename='https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network.png')

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network.png">

In [ ]:
review = "The movie was excellent"

#Image(filename='sentiment_network_pos.png')

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network_pos.png">

# Project 2: Creating the Input/Output Data<a id='project_2'></a>

**TODO:** Create a [set](https://docs.python.org/3/tutorial/datastructures.html#sets) named `vocab` that contains every word in the vocabulary.

In [ ]:
# TODO: Create set named "vocab" containing all of the words from all of the reviews

#All the words are already in the total_counts Counter , i dont need to go to every review
vocab = set(total_counts.keys())

Run the following cell to check your vocabulary size. If everything worked correctly, it should print **74074**

In [ ]:
vocab_size = len(vocab)
print(vocab_size)

74074


Take a look at the following image. It represents the layers of the neural network you'll be building throughout this notebook. `layer_0` is the input layer, `layer_1` is a hidden layer, and `layer_2` is the output layer.

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network_2.png">

**TODO:** Create a numpy array called `layer_0` and initialize it to all zeros. You will find the [zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) function particularly helpful here. Be sure you create `layer_0` as a 2-dimensional matrix with 1 row and `vocab_size` columns. 

In [ ]:
# TODO: Create layer_0 matrix with dimensions 1 by vocab_size, initially filled with zeros
layer_0 = np.array(np.zeros((1,vocab_size)))

Run the following cell. It should display `(1, 74074)`

In [ ]:
layer_0.shape

(1, 74074)

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network.png">

`layer_0` contains one entry for every word in the vocabulary, as shown in the above image. We need to make sure we know the index of each word, so run the following cell to create a lookup table that stores the index of every word.

In [ ]:
# Create a dictionary of words in the vocabulary mapped to index positions
# (to be used in layer_0)
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
    
# display the map of words to indices
word2index

{'': 0,
 'rookies': 1,
 'charade': 2,
 'endorsed': 3,
 'advocacy': 4,
 'spoliers': 5,
 'salaam': 6,
 'mabuse': 7,
 'awww': 8,
 'conpiricy': 9,
 'jindabyne': 10,
 'ballgames': 11,
 'basket': 12,
 'continued': 13,
 'elizabethan': 14,
 'ghungroo': 15,
 'baffle': 16,
 'camra': 17,
 'abhays': 18,
 'luana': 19,
 'hashmi': 20,
 'mvp': 21,
 'unoriginality': 22,
 'highly': 23,
 'detatched': 24,
 'wilkinson': 25,
 'absurdness': 26,
 'viability': 27,
 'chickboxer': 28,
 'roget': 29,
 'teens': 30,
 'villainous': 31,
 'approval': 32,
 'demagogue': 33,
 'egyptologistic': 34,
 'johnstone': 35,
 'brechtian': 36,
 'fuing': 37,
 'tyrants': 38,
 'intrinsically': 39,
 'loooooooooooong': 40,
 'spoiler': 41,
 'poland': 42,
 'independents': 43,
 'highlighting': 44,
 'vilifies': 45,
 'dinosaur': 46,
 'bejesus': 47,
 'slicing': 48,
 'saree': 49,
 'geishas': 50,
 'weathered': 51,
 'liana': 52,
 'sarafian': 53,
 'holst': 54,
 'apparel': 55,
 'kristoffersons': 56,
 'store': 57,
 'soh': 58,
 'kerosene': 59,
 'inse

**TODO:**  Complete the implementation of `update_input_layer`. It should count 
          how many times each word is used in the given review, and then store
          those counts at the appropriate indices inside `layer_0`.

In [ ]:
def update_input_layer(review):
    """ Modify the global layer_0 to represent the vector form of review.
    The element at a given index of layer_0 should represent
    how many times the given word occurs in the review.
    Args:
        review(string) - the string of the review
    Returns:
        None
    """
    global layer_0
    # clear out previous state by resetting the layer to be all 0s
    layer_0 *= 0
    
    # TODO: count how many times each word is used in the given review and store the results in layer_0
    counter = Counter(review.split(' '))
    for word in counter:
        word_index = word2index[word]
        layer_0[0,word_index] = counter[word]

Run the following cell to test updating the input layer with the first review. The indices assigned may not be the same as in the solution, but hopefully you'll see some non-zero values in `layer_0`.  

In [ ]:
update_input_layer(reviews[0])
layer_0

array([[18.,  0.,  0., ...,  0.,  0.,  0.]])

**TODO:** Complete the implementation of `get_target_for_labels`. It should return `0` or `1`, 
          depending on whether the given label is `NEGATIVE` or `POSITIVE`, respectively.

In [ ]:
def get_target_for_label(label):
    """Convert a label to `0` or `1`.
    Args:
        label(string) - Either "POSITIVE" or "NEGATIVE".
    Returns:
        `0` or `1`.
    """
    # TODO: Your code here
    return 0 if label == 'NEGATIVE' else 1

Run the following two cells. They should print out`'POSITIVE'` and `1`, respectively.

In [ ]:
labels[0]

'POSITIVE'

In [ ]:
get_target_for_label(labels[0])

1

Run the following two cells. They should print out `'NEGATIVE'` and `0`, respectively.

In [ ]:
labels[1]

'NEGATIVE'

In [ ]:
get_target_for_label(labels[1])

0

# End of Project 2. 


# Project 3: Building a Neural Network<a id='project_3'></a>

Antes de entrar a detalles, veamos un modelo similar en keras

In [ ]:
import tensorflow as tf

red_neuronal = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units = 10,activation="sigmoid",kernel_initializer=tf.keras.initializers.GlorotNormal()),
    tf.keras.layers.Dense(units = 1,activation="sigmoid",kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01))
])

In [ ]:
red_neuronal.compile(optimizer="sgd",loss="categorical_crossentropy")

**TODO:** We've included the framework of a class called `SentimentNetork`. Implement all of the items marked `TODO` in the code. These include doing the following:
- Create a basic neural network much like the networks you've seen in earlier lessons and in Project 1, with an input layer, a hidden layer, and an output layer. 
- Do **not** add a non-linearity in the hidden layer. That is, do not use an activation function when calculating the hidden layer outputs.
- Re-use the code from earlier in this notebook to create the training data (see `TODO`s in the code)
- Implement the `pre_process_data` function to create the vocabulary for our training data generating functions
- Ensure `train` trains over the entire corpus

In [ ]:
import time
import sys
import numpy as np

# Encapsulate our neural network in a class
class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 10, learning_rate = 0.1):
        """Create a SentimenNetwork with the given settings
        Args:
            reviews(list) - List of reviews used for training
            labels(list) - List of POSITIVE/NEGATIVE labels associated with the given reviews
            hidden_nodes(int) - Number of nodes to create in the hidden layer
            learning_rate(float) - Learning rate to use while training
        
        """
        # Assign a seed to our random number generator to ensure we get
        # reproducable results during development 
        np.random.seed(1)

        # process the reviews and their associated labels so that everything
        # is ready for training
        self.pre_process_data(reviews, labels)
        
        # Build the network to have the number of hidden nodes and the learning rate that
        # were passed into this initializer. Make the same number of input nodes as
        # there are vocabulary words and create a single output node.
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
        
        review_vocab = set()
        # TODO: populate review_vocab with all of the words in the given reviews
        #       Remember to split reviews into individual words 
        #       using "split(' ')" instead of "split()".
        
        for review in reviews:
            words = review.split(' ')
            for word in words:
                review_vocab.add(word)
        # Convert the vocabulary set to a list so we can access words via indices
        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        # TODO: populate label_vocab with all of the words in the given labels.
        #       There is no need to split the labels because each one is a single word.
        
        for label in labels:
            label_vocab.add(label)
        # Convert the label vocabulary set to a list so we can access labels via indices
        self.label_vocab = list(label_vocab)
        
        # Store the sizes of the review and label vocabularies.
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        # Create a dictionary of words in the vocabulary mapped to index positions
        self.word2index = {}
        # TODO: populate self.word2index with indices for all the words in self.review_vocab
        #       like you saw earlier in the notebook
        for index,word in enumerate(self.review_vocab):
            self.word2index[word] = index
        # Create a dictionary of labels mapped to index positions
        self.label2index = {}
        # TODO: do the same thing you did for self.word2index and self.review_vocab, 
        #       but for self.label2index and self.label_vocab instead
        for index,label in enumerate(self.label_vocab):
            self.label2index[label] = index
            
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Store the number of nodes in input, hidden, and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Store the learning rate
        self.learning_rate = learning_rate

        # Initialize weights
        
        # TODO: initialize self.weights_0_1 as a matrix of zeros. These are the weights between
        #       the input layer and the hidden layer.
        self.weights_0_1 = np.random.normal(0.0, np.sqrt(1/input_nodes),
                                            (input_nodes,hidden_nodes))
        
        # TODO: initialize self.weights_1_2 as a matrix of random values. 
        #       These are the weights between the hidden layer and the output layer.
        self.weights_1_2 = np.random.normal(0.0, np.sqrt(1/hidden_nodes), 
                                                (self.hidden_nodes, self.output_nodes))
        
        # TODO: Create the input layer, a two-dimensional matrix with shape 
        #       1 x input_nodes, with all values initialized to zero
        self.layer_0 = np.zeros((1,input_nodes))
    
        
    def update_input_layer(self,review):
        # TODO: You can copy most of the code you wrote for update_input_layer 
        #       earlier in this notebook. 
        #
        #       However, MAKE SURE YOU CHANGE ALL VARIABLES TO REFERENCE
        #       THE VERSIONS STORED IN THIS OBJECT, NOT THE GLOBAL OBJECTS.
        #       For example, replace "layer_0 *= 0" with "self.layer_0 *= 0"
        self.layer_0*=0
        counter = Counter(review.split(' '))
        for word in counter:
            if word in self.word2index.keys():
                word_count = counter[word]
                word_index = self.word2index[word]
                self.layer_0[0,word_index] = word_count
                
    def get_target_for_label(self,label):
        # TODO: Copy the code you wrote for get_target_for_label 
        #       earlier in this notebook. 
        return 0 if label == 'NEGATIVE' else 1
        
    def sigmoid(self,x):
        # TODO: Return the result of calculating the sigmoid activation function
        #       shown in the lectures
        return 1 / ( 1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        # TODO: Return the derivative of the sigmoid activation function, 
        #       where "output" is the original output from the sigmoid fucntion 
        return output * (1 - output)

    def sigmoid_prime(self,output):
        # TODO: Return the derivative of the sigmoid activation function, 
        return output * (1 - output)


    def train(self, training_reviews, training_labels):
        
        # make sure out we have a matching number of reviews and labels
        assert(len(training_reviews) == len(training_labels))
        
        # Keep track of correct predictions to display accuracy during training 
        correct_so_far = 0
        
        # Remember when we started for printing time statistics
        start = time.time()

        # loop through all the given reviews and run a forward and backward pass,
        # updating weights for every item
        for i in range(len(training_reviews)):
            
            # TODO: Get the next review and its correct label
            review = training_reviews[i]
            label = training_labels[i]
            # TODO: Implement the forward pass through the network. 
            #       That means use the given review to update the input layer, 
            #       then calculate values for the hidden layer,
            #       and finally calculate the output layer.
            # 
            #       Do not use an activation function for the hidden layer,
            #       but use the sigmoid activation function for the output layer.
            self.update_input_layer(review)
            hidden_layer_logits = np.dot(self.layer_0,self.weights_0_1)
            hidden_layer_output = self.sigmoid(hidden_layer_logits)
            output_layer_logits = np.dot(hidden_layer_output,self.weights_1_2)
            output_layer_predictions = self.sigmoid(output_layer_logits)
            #print("output layer",output_layer_predictions.shape)
            # TODO: Implement the back propagation pass here. 
            #       That means calculate the error for the forward pass's prediction
            #       and update the weights in the network according to their
            #       contributions toward the error, as calculated via the
            #       gradient descent and back propagation algorithms you 
            #       learned in class.
            error = output_layer_predictions - self.get_target_for_label(label) # dL/da2(derivada del costo respecto de la salida y_pred)
            #print("error",error.shape)
            #calculate the error terms of each layer

            #dL/dz2(derivada del costo respecto de la pre-activacion z2, el termino de error delta2)
            output_error_term = error*self.sigmoid_prime(output_layer_predictions) 
            #print("output error term",output_error_term.shape)
            #print("sigmoid prime(hidden)",self.sigmoid_prime(hidden_layer_output).shape)

            # dL/dz1 (derivada del costo respecto de la pre-activacion z1, el termino de error delta1)
            hidden_error_term = np.matmul(self.weights_1_2,output_error_term)*self.sigmoid_prime(hidden_layer_output).T

            ## una vez calculados los terminos de error delta(derivadas respecto de z) los usamos para calcular las derivadas parciales respecto de los parametros usando #activacion entrante * error saliente
            grad_weights_h_o =  output_error_term * hidden_layer_output
            grad_weights_i_h = hidden_error_term * self.layer_0

            ## una vez calculadas las derivadas respecto de los parametros aplicamos "gradient descent"
            self.weights_1_2-= (self.learning_rate*grad_weights_h_o).T
            self.weights_0_1-= (self.learning_rate*grad_weights_i_h).T
            # TODO: Keep track of correct predictions. To determine if the prediction was
            #       correct, check that the absolute value of the output error 
            #       is less than 0.5. If so, add one to the correct_so_far count.
            #Note to myself: strange way to calc correctness hmmm
            if  np.abs(error) < 0.5:
                correct_so_far+=1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given review.
        """
        # TODO: Run a forward pass through the network, like you did in the
        #       "train" function. That means use the given review to 
        #       update the input layer, then calculate values for the hidden layer,
        #       and finally calculate the output layer.
        #
        #       Note: The review passed into this function for prediction 
        #             might come from anywhere, so you should convert it 
        #             to lower case prior to using it.
        review = review.lower()
        self.update_input_layer(review)
        hidden_logits = np.dot(self.layer_0,self.weights_0_1)
        hidden_output = self.sigmoid(hidden_logits)
        output_logits = np.dot(hidden_output,self.weights_1_2)
        output = self.sigmoid(output_logits)

        # TODO: The output layer should now contain a prediction. 
        #       Return `POSITIVE` for predictions greater-than-or-equal-to `0.5`, 
        #       and `NEGATIVE` otherwise.
        return "POSITIVE" if output >= 0.5 else "NEGATIVE"


Run the following cell to create a `SentimentNetwork` that will train on all but the last 1000 reviews (we're saving those for testing). Here we use a learning rate of `0.1`.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)

Run the following cell to test the network's performance against the last 1000 reviews (the ones we held out from our training set). 

**We have not trained the model yet, so the results should be about 50% as it will just be guessing and there are only two possible values to choose from.**

In [ ]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):612.6 #Correct:500 #Tested:1000 Testing Accuracy:50.0%

Run the following cell to actually train the network. During training, it will display the model's accuracy repeatedly as it trains so you can see how well it's doing.

In [ ]:
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:10.4% Speed(reviews/sec):163.7 #Correct:1284 #Trained:2501 Training Accuracy:51.3%
Progress:20.8% Speed(reviews/sec):163.5 #Correct:2628 #Trained:5001 Training Accuracy:52.5%
Progress:31.2% Speed(reviews/sec):162.6 #Correct:4045 #Trained:7501 Training Accuracy:53.9%
Progress:41.6% Speed(reviews/sec):162.9 #Correct:5540 #Trained:10001 Training Accuracy:55.3%
Progress:52.0% Speed(reviews/sec):163.1 #Correct:7064 #Trained:12501 Training Accuracy:56.5%
Progress:62.4% Speed(reviews/sec):163.2 #Correct:8585 #Trained:15001 Training Accuracy:57.2%
Progress:72.9% Speed(reviews/sec):163.2 #Correct:9989 #Trained:17501 Training Accuracy:57.0%
Progress:83.3% Speed(reviews/sec):163.2 #Correct:11524 #Trained:20001 Training Accuracy:57.6%
Progress:93.7% Speed(reviews/sec):163.0 #Correct:13001 #Trained:22501 Training Accuracy:57.7%
Progress:99.9% Speed(reviews/sec):163.1 #Correct:13935 #Trained:24001 Training Acc

That most likely didn't train very well. Part of the reason may be because the learning rate is too high. Run the following cell to recreate the network with a smaller learning rate, `0.01`, and then train the new network.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.01)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:10.4% Speed(reviews/sec):154.0 #Correct:1356 #Trained:2501 Training Accuracy:54.2%
Progress:20.8% Speed(reviews/sec):156.8 #Correct:2877 #Trained:5001 Training Accuracy:57.5%
Progress:31.2% Speed(reviews/sec):159.0 #Correct:4471 #Trained:7501 Training Accuracy:59.6%
Progress:41.6% Speed(reviews/sec):160.5 #Correct:6161 #Trained:10001 Training Accuracy:61.6%
Progress:52.0% Speed(reviews/sec):161.1 #Correct:7885 #Trained:12501 Training Accuracy:63.0%
Progress:62.4% Speed(reviews/sec):161.1 #Correct:9626 #Trained:15001 Training Accuracy:64.1%
Progress:72.9% Speed(reviews/sec):161.3 #Correct:11385 #Trained:17501 Training Accuracy:65.0%
Progress:83.3% Speed(reviews/sec):161.4 #Correct:13219 #Trained:20001 Training Accuracy:66.0%
Progress:93.7% Speed(reviews/sec):161.4 #Correct:15046 #Trained:22501 Training Accuracy:66.8%
Progress:99.9% Speed(reviews/sec):161.3 #Correct:16182 #Trained:24001 Training Ac

That probably wasn't much different. Run the following cell to recreate the network one more time with an even smaller learning rate, `0.001`, and then train the new network.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.001)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:10.4% Speed(reviews/sec):149.3 #Correct:1321 #Trained:2501 Training Accuracy:52.8%
Progress:20.8% Speed(reviews/sec):150.3 #Correct:2724 #Trained:5001 Training Accuracy:54.4%
Progress:31.2% Speed(reviews/sec):151.0 #Correct:4212 #Trained:7501 Training Accuracy:56.1%
Progress:41.6% Speed(reviews/sec):151.9 #Correct:5709 #Trained:10001 Training Accuracy:57.0%
Progress:52.0% Speed(reviews/sec):152.5 #Correct:7283 #Trained:12501 Training Accuracy:58.2%
Progress:62.4% Speed(reviews/sec):152.8 #Correct:8826 #Trained:15001 Training Accuracy:58.8%
Progress:72.9% Speed(reviews/sec):153.1 #Correct:10397 #Trained:17501 Training Accuracy:59.4%
Progress:83.3% Speed(reviews/sec):153.0 #Correct:12014 #Trained:20001 Training Accuracy:60.0%
Progress:93.7% Speed(reviews/sec):153.2 #Correct:13684 #Trained:22501 Training Accuracy:60.8%
Progress:99.9% Speed(reviews/sec):153.1 #Correct:14713 #Trained:24001 Training Ac

With a learning rate of `0.001`, the network should finall have started to improve during training. It's still not very good, but it shows that this solution has potential. We will improve it in the next lesson.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.0003)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:10.4% Speed(reviews/sec):144.5 #Correct:1266 #Trained:2501 Training Accuracy:50.6%
Progress:20.8% Speed(reviews/sec):149.6 #Correct:2572 #Trained:5001 Training Accuracy:51.4%
Progress:31.2% Speed(reviews/sec):152.0 #Correct:3945 #Trained:7501 Training Accuracy:52.5%
Progress:41.6% Speed(reviews/sec):152.7 #Correct:5328 #Trained:10001 Training Accuracy:53.2%
Progress:52.0% Speed(reviews/sec):152.3 #Correct:6757 #Trained:12501 Training Accuracy:54.0%
Progress:62.4% Speed(reviews/sec):152.2 #Correct:8182 #Trained:15001 Training Accuracy:54.5%
Progress:72.9% Speed(reviews/sec):152.2 #Correct:9609 #Trained:17501 Training Accuracy:54.9%
Progress:83.3% Speed(reviews/sec):152.7 #Correct:11113 #Trained:20001 Training Accuracy:55.5%
Progress:93.7% Speed(reviews/sec):153.0 #Correct:12600 #Trained:22501 Training Accuracy:55.9%
Progress:99.9% Speed(reviews/sec):153.3 #Correct:13549 #Trained:24001 Training Acc

# End of Project 3. 

# Understanding Neural Noise<a id='lesson_4'></a>


<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network.png">

In [ ]:
def update_input_layer(review):
    
    global layer_0
    
    # clear out previous state, reset the layer to be all 0s
    layer_0 *= 0
    for word in review.split(" "):
        layer_0[0][word2index[word]] += 1

update_input_layer(reviews[0])

In [ ]:
layer_0

array([[18.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
review_counter = Counter()

In [ ]:
for word in reviews[0].split(" "):
    review_counter[word] += 1

In [ ]:
review_counter.most_common()

[('.', 27),
 ('', 18),
 ('the', 9),
 ('to', 6),
 ('high', 5),
 ('i', 5),
 ('bromwell', 4),
 ('is', 4),
 ('a', 4),
 ('teachers', 4),
 ('that', 4),
 ('of', 4),
 ('it', 2),
 ('at', 2),
 ('as', 2),
 ('school', 2),
 ('my', 2),
 ('in', 2),
 ('me', 2),
 ('students', 2),
 ('their', 2),
 ('student', 2),
 ('cartoon', 1),
 ('comedy', 1),
 ('ran', 1),
 ('same', 1),
 ('time', 1),
 ('some', 1),
 ('other', 1),
 ('programs', 1),
 ('about', 1),
 ('life', 1),
 ('such', 1),
 ('years', 1),
 ('teaching', 1),
 ('profession', 1),
 ('lead', 1),
 ('believe', 1),
 ('s', 1),
 ('satire', 1),
 ('much', 1),
 ('closer', 1),
 ('reality', 1),
 ('than', 1),
 ('scramble', 1),
 ('survive', 1),
 ('financially', 1),
 ('insightful', 1),
 ('who', 1),
 ('can', 1),
 ('see', 1),
 ('right', 1),
 ('through', 1),
 ('pathetic', 1),
 ('pomp', 1),
 ('pettiness', 1),
 ('whole', 1),
 ('situation', 1),
 ('all', 1),
 ('remind', 1),
 ('schools', 1),
 ('knew', 1),
 ('and', 1),
 ('when', 1),
 ('saw', 1),
 ('episode', 1),
 ('which', 1),
 ('r

# Project 4: Reducing Noise in Our Input Data<a id='project_4'></a>

**TODO:** Attempt to reduce the noise in the input data like . Specifically, do the following:
* Copy the `SentimentNetwork` class you created earlier into the following cell.
* Modify `update_input_layer` so it does not count how many times each word is used, but rather just stores whether or not a word was used. 

In [ ]:
# TODO: -Copy the SentimentNetwork class from Projet 3 lesson
#       -Modify it to reduce noise by setting 1(presence) or 0(ausence) instead of counts
import time
import sys
import numpy as np

# Encapsulate our neural network in a class
class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 10, learning_rate = 0.1):
        """Create a SentimenNetwork with the given settings
        Args:
            reviews(list) - List of reviews used for training
            labels(list) - List of POSITIVE/NEGATIVE labels associated with the given reviews
            hidden_nodes(int) - Number of nodes to create in the hidden layer
            learning_rate(float) - Learning rate to use while training
        
        """
        # Assign a seed to our random number generator to ensure we get
        # reproducable results during development 
        np.random.seed(1)

        # process the reviews and their associated labels so that everything
        # is ready for training
        self.pre_process_data(reviews, labels)
        
        # Build the network to have the number of hidden nodes and the learning rate that
        # were passed into this initializer. Make the same number of input nodes as
        # there are vocabulary words and create a single output node.
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
        
        review_vocab = set()
        # TODO: populate review_vocab with all of the words in the given reviews
        #       Remember to split reviews into individual words 
        #       using "split(' ')" instead of "split()".
        
        for review in reviews:
            words = review.split(' ')
            for word in words:
                review_vocab.add(word)
        # Convert the vocabulary set to a list so we can access words via indices
        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        # TODO: populate label_vocab with all of the words in the given labels.
        #       There is no need to split the labels because each one is a single word.
        
        for label in labels:
            label_vocab.add(label)
        # Convert the label vocabulary set to a list so we can access labels via indices
        self.label_vocab = list(label_vocab)
        
        # Store the sizes of the review and label vocabularies.
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        # Create a dictionary of words in the vocabulary mapped to index positions
        self.word2index = {}
        # TODO: populate self.word2index with indices for all the words in self.review_vocab
        #       like you saw earlier in the notebook
        for index,word in enumerate(self.review_vocab):
            self.word2index[word] = index
        # Create a dictionary of labels mapped to index positions
        self.label2index = {}
        # TODO: do the same thing you did for self.word2index and self.review_vocab, 
        #       but for self.label2index and self.label_vocab instead
        for index,label in enumerate(self.label_vocab):
            self.label2index[label] = index
            
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Store the number of nodes in input, hidden, and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Store the learning rate
        self.learning_rate = learning_rate

        # Initialize weights
        
        # TODO: initialize self.weights_0_1 as a matrix of zeros. These are the weights between
        #       the input layer and the hidden layer.
        self.weights_0_1 = np.random.normal(0.0, np.sqrt(1/input_nodes),
                                            (input_nodes,hidden_nodes))
        
        # TODO: initialize self.weights_1_2 as a matrix of random values. 
        #       These are the weights between the hidden layer and the output layer.
        self.weights_1_2 = np.random.normal(0.0, np.sqrt(1/input_nodes), 
                                                (self.hidden_nodes, self.output_nodes))
        
        # TODO: Create the input layer, a two-dimensional matrix with shape 
        #       1 x input_nodes, with all values initialized to zero
        self.layer_0 = np.zeros((1,input_nodes))
    
        
    def update_input_layer(self,review):
        # TODO: You can copy most of the code you wrote for update_input_layer 
        #       earlier in this notebook. 
        #
        #       However, MAKE SURE YOU CHANGE ALL VARIABLES TO REFERENCE
        #       THE VERSIONS STORED IN THIS OBJECT, NOT THE GLOBAL OBJECTS.
        #       For example, replace "layer_0 *= 0" with "self.layer_0 *= 0"
        self.layer_0*=0
        for word in review.split(' '):
            if word in self.word2index.keys():
                word_index = self.word2index[word]
                self.layer_0[0,word_index] = 1
                
    def get_target_for_label(self,label):
        # TODO: Copy the code you wrote for get_target_for_label 
        #       earlier in this notebook. 
        return 0 if label == 'NEGATIVE' else 1
        
    def sigmoid(self,x):
        # TODO: Return the result of calculating the sigmoid activation function
        #       shown in the lectures
        return 1 / ( 1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        # TODO: Return the derivative of the sigmoid activation function, 
        #       where "output" is the original output from the sigmoid fucntion 
        return output * (1 - output)

    def sigmoid_prime(self,output):
        # TODO: Return the derivative of the sigmoid activation function, 
        return output * (1 - output)


    def train(self, training_reviews, training_labels):
        
        # make sure out we have a matching number of reviews and labels
        assert(len(training_reviews) == len(training_labels))
        
        # Keep track of correct predictions to display accuracy during training 
        correct_so_far = 0
        
        # Remember when we started for printing time statistics
        start = time.time()

        # loop through all the given reviews and run a forward and backward pass,
        # updating weights for every item
        for i in range(len(training_reviews)):
            
            # TODO: Get the next review and its correct label
            review = training_reviews[i]
            label = training_labels[i]
            # TODO: Implement the forward pass through the network. 
            #       That means use the given review to update the input layer, 
            #       then calculate values for the hidden layer,
            #       and finally calculate the output layer.
            # 
            #       Do not use an activation function for the hidden layer,
            #       but use the sigmoid activation function for the output layer.
            self.update_input_layer(review)
            hidden_layer_logits = np.dot(self.layer_0,self.weights_0_1)
            hidden_layer_output = self.sigmoid(hidden_layer_logits)
            output_layer_logits = np.dot(hidden_layer_output,self.weights_1_2)
            output_layer_predictions = self.sigmoid(output_layer_logits)
            #print("output layer",output_layer_predictions.shape)
            # TODO: Implement the back propagation pass here. 
            #       That means calculate the error for the forward pass's prediction
            #       and update the weights in the network according to their
            #       contributions toward the error, as calculated via the
            #       gradient descent and back propagation algorithms you 
            #       learned in class.
            error = output_layer_predictions - self.get_target_for_label(label) # dL/da2(derivada del costo respecto de la salida y_pred)
            #print("error",error.shape)
            #calculate the error terms of each layer
            output_error_term = error*self.sigmoid_prime(output_layer_predictions) #dL/dz2(derivada del costo respecto de la pre-activacion z2, el termino de error delta2)
            #print("output error term",output_error_term.shape)
            #print("sigmoid prime(hidden)",self.sigmoid_prime(hidden_layer_output).shape)
            hidden_error_term = np.matmul(self.weights_1_2,output_error_term)*self.sigmoid_prime(hidden_layer_output).T# dL/dz1 (derivada del costo respecto de la pre-activacion z1, el termino de error delta1)
            #print("hidden error term",hidden_error_term.shape)
            #print("hidden layer output",hidden_layer_output.shape)
            #calculate the delta on the weights of each layer

            ## una vez calculados los terminos de error delta(derivadas respecto de z) los usamos para calcular las derivadas parciales respecto de los parametros usando #activacion entrante * error saliente
            grad_weights_h_o =  output_error_term * hidden_layer_output
            grad_weights_i_h = hidden_error_term * self.layer_0
            #print("calculated",(self.learning_rate*delta_weights_h_o).shape)
            #print("weights",self.weights_1_2.shape)

            ## una vez calculadas las derivadas respecto de los parametros aplicamos "gradient descent"
            self.weights_1_2-= (self.learning_rate*grad_weights_h_o).T
            self.weights_0_1-= (self.learning_rate*grad_weights_i_h).T
            # TODO: Keep track of correct predictions. To determine if the prediction was
            #       correct, check that the absolute value of the output error 
            #       is less than 0.5. If so, add one to the correct_so_far count.
            #Note to myself: strange way to calc correctness hmmm
            if  np.abs(error) < 0.5:
                correct_so_far+=1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given review.
        """
        # TODO: Run a forward pass through the network, like you did in the
        #       "train" function. That means use the given review to 
        #       update the input layer, then calculate values for the hidden layer,
        #       and finally calculate the output layer.
        #
        #       Note: The review passed into this function for prediction 
        #             might come from anywhere, so you should convert it 
        #             to lower case prior to using it.
        review = review.lower()
        self.update_input_layer(review)
        hidden_logits = np.dot(self.layer_0,self.weights_0_1)
        hidden_output = self.sigmoid(hidden_logits)
        output_logits = np.dot(hidden_output,self.weights_1_2)
        output = self.sigmoid(output_logits)

        # TODO: The output layer should now contain a prediction. 
        #       Return `POSITIVE` for predictions greater-than-or-equal-to `0.5`, 
        #       and `NEGATIVE` otherwise.
        return "POSITIVE" if output >= 0.5 else "NEGATIVE"


Run the following cell to recreate the network and train it. Notice we've gone back to the higher learning rate of `0.1`.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:10.4% Speed(reviews/sec):152.1 #Correct:587 #Trained:2501 Training Accuracy:23.4%
Progress:20.8% Speed(reviews/sec):151.7 #Correct:2496 #Trained:5001 Training Accuracy:49.9%
Progress:31.2% Speed(reviews/sec):153.0 #Correct:4575 #Trained:7501 Training Accuracy:60.9%
Progress:41.6% Speed(reviews/sec):153.3 #Correct:6715 #Trained:10001 Training Accuracy:67.1%
Progress:52.0% Speed(reviews/sec):153.2 #Correct:8849 #Trained:12501 Training Accuracy:70.7%
Progress:62.4% Speed(reviews/sec):153.2 #Correct:10992 #Trained:15001 Training Accuracy:73.2%
Progress:72.9% Speed(reviews/sec):153.3 #Correct:13094 #Trained:17501 Training Accuracy:74.8%
Progress:83.3% Speed(reviews/sec):153.6 #Correct:15255 #Trained:20001 Training Accuracy:76.2%
Progress:93.7% Speed(reviews/sec):154.1 #Correct:17440 #Trained:22501 Training Accuracy:77.5%
Progress:99.9% Speed(reviews/sec):154.5 #Correct:18760 #Trained:24001 Training Ac

That should have trained much better than the earlier attempts. It's still not wonderful, but it should have improved dramatically. Run the following cell to test your model with 1000 predictions.

In [ ]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):598.2 #Correct:846 #Tested:1000 Testing Accuracy:84.6%

# End of Project 4. 

# Analyzing Inefficiencies in our Network<a id='lesson_5'></a>

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network_sparse.png">

In [ ]:
layer_0 = np.zeros(10)

In [ ]:
layer_0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
layer_0[4] = 1
layer_0[9] = 1

In [ ]:
layer_0

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.])

In [ ]:
weights_0_1 = np.random.randn(10,5)

In [ ]:
layer_0.dot(weights_0_1)

array([ 2.40385671, -2.03143821, -1.01001659, -0.20106613,  2.10673993])

In [ ]:
indices = [4,9]

In [ ]:
layer_1 = np.zeros(5)

In [ ]:
for index in indices:
    layer_1 += (1 * weights_0_1[index])

In [ ]:
layer_1

array([ 2.40385671, -2.03143821, -1.01001659, -0.20106613,  2.10673993])

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network_sparse_2.png">

In [ ]:
layer_1 = np.zeros(5)

In [ ]:
for index in indices:
    layer_1 += (weights_0_1[index])

In [ ]:
layer_1

array([ 2.40385671, -2.03143821, -1.01001659, -0.20106613,  2.10673993])

# Project 5: Making our Network More Efficient<a id='project_5'></a>
**TODO:** Make the `SentimentNetwork` class more efficient by eliminating unnecessary multiplications and additions that occur during forward and backward propagation. To do that, you can do the following:
* Copy the `SentimentNetwork` class from the previous project into the following cell.
* Remove the `update_input_layer` function - you will not need it in this version.
* Modify `init_network`:
>* You no longer need a separate input layer, so remove any mention of `self.layer_0`
>* You will be dealing with the old hidden layer more directly, so create `self.layer_1`, a two-dimensional matrix with shape 1 x hidden_nodes, with all values initialized to zero
* Modify `train`:
>* Change the name of the input parameter `training_reviews` to `training_reviews_raw`. This will help with the next step.
>* At the beginning of the function, you'll want to preprocess your reviews to convert them to a list of indices (from `word2index`) that are actually used in the review. This is equivalent to what you saw in the video when Andrew set specific indices to 1. Your code should create a local `list` variable named `training_reviews` that should contain a `list` for each review in `training_reviews_raw`. Those lists should contain the indices for words found in the review.
>* Remove call to `update_input_layer`
>* Use `self`'s  `layer_1` instead of a local `layer_1` object.
>* In the forward pass, replace the code that updates `layer_1` with new logic that only adds the weights for the indices used in the review.
>* When updating `weights_0_1`, only update the individual weights that were used in the forward pass.
* Modify `run`:
>* Remove call to `update_input_layer` 
>* Use `self`'s  `layer_1` instead of a local `layer_1` object.
>* Much like you did in `train`, you will need to pre-process the `review` so you can work with word indices, then update `layer_1` by adding weights for the indices used in the review.

In [ ]:
# TODO: -Copy the SentimentNetwork class from Project 4 lesson
#       -Modify it according to the above instructions 
#       -Modify it to reduce noise, like in the video 
import time
import sys
import numpy as np

# Encapsulate our neural network in a class
class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 10, learning_rate = 0.1):
        """Create a SentimenNetwork with the given settings
        Args:
            reviews(list) - List of reviews used for training
            labels(list) - List of POSITIVE/NEGATIVE labels associated with the given reviews
            hidden_nodes(int) - Number of nodes to create in the hidden layer
            learning_rate(float) - Learning rate to use while training
        
        """
        # Assign a seed to our random number generator to ensure we get
        # reproducable results during development 
        np.random.seed(1)

        # process the reviews and their associated labels so that everything
        # is ready for training
        self.pre_process_data(reviews, labels)
        
        # Build the network to have the number of hidden nodes and the learning rate that
        # were passed into this initializer. Make the same number of input nodes as
        # there are vocabulary words and create a single output node.
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
        
        review_vocab = set()
        # TODO: populate review_vocab with all of the words in the given reviews
        #       Remember to split reviews into individual words 
        #       using "split(' ')" instead of "split()".
        
        for review in reviews:
            words = review.split(' ')
            for word in words:
                review_vocab.add(word)
        # Convert the vocabulary set to a list so we can access words via indices
        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        # TODO: populate label_vocab with all of the words in the given labels.
        #       There is no need to split the labels because each one is a single word.
        
        for label in labels:
            label_vocab.add(label)
        # Convert the label vocabulary set to a list so we can access labels via indices
        self.label_vocab = list(label_vocab)
        
        # Store the sizes of the review and label vocabularies.
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        # Create a dictionary of words in the vocabulary mapped to index positions
        self.word2index = {}
        # TODO: populate self.word2index with indices for all the words in self.review_vocab
        #       like you saw earlier in the notebook
        for index,word in enumerate(self.review_vocab):
            self.word2index[word] = index
        # Create a dictionary of labels mapped to index positions
        self.label2index = {}
        # TODO: do the same thing you did for self.word2index and self.review_vocab, 
        #       but for self.label2index and self.label_vocab instead
        for index,label in enumerate(self.label_vocab):
            self.label2index[label] = index
            
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Store the number of nodes in input, hidden, and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Store the learning rate
        self.learning_rate = learning_rate

        # Initialize weights
        
        # TODO: initialize self.weights_0_1 as a matrix of zeros. These are the weights between
        #       the input layer and the hidden layer.
        self.weights_0_1 = np.zeros((input_nodes,hidden_nodes))
        
        # TODO: initialize self.weights_1_2 as a matrix of random values. 
        #       These are the weights between the hidden layer and the output layer.
        self.weights_1_2 = np.random.normal(0.0, self.output_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        # TODO: Create the input layer, a two-dimensional matrix with shape 
        #       1 x input_nodes, with all values initialized to zero
        self.layer_1 = np.zeros((1,hidden_nodes))
    
                
    def get_target_for_label(self,label):
        # TODO: Copy the code you wrote for get_target_for_label 
        #       earlier in this notebook. 
        return 0 if label == 'NEGATIVE' else 1
        
    def sigmoid(self,x):
        # TODO: Return the result of calculating the sigmoid activation function
        #       shown in the lectures
        return 1 / ( 1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        # TODO: Return the derivative of the sigmoid activation function, 
        #       where "output" is the original output from the sigmoid fucntion 
        return output * (1 - output)

    def train(self, training_reviews_raw, training_labels):
        
        # make sure out we have a matching number of reviews and labels
        assert(len(training_reviews_raw) == len(training_labels))
        #create a list of lists, each inner list contains the word2index index for a review
        training_reviews = list()
        

        for review in training_reviews_raw:
            review_indices = set()
            words = review.split(' ')
            for word in words:
                if word in self.word2index:
                    review_indices.add(self.word2index[word])
            training_reviews.append(list(review_indices))
        # Keep track of correct predictions to display accuracy during training 
        correct_so_far = 0
        
        # Remember when we started for printing time statistics
        start = time.time()

        # loop through all the given reviews and run a forward and backward pass,
        # updating weights for every item
        for i in range(len(training_reviews)):
            
            # TODO: Get the next review and its correct label
            review = training_reviews[i]
            label = training_labels[i]
            # TODO: Implement the forward pass through the network. 
            #       That means use the given review to update the input layer, 
            #       then calculate values for the hidden layer,
            #       and finally calculate the output layer.
            # 
            #       Do not use an activation function for the hidden layer,
            #       but use the sigmoid activation function for the output layer.
            
            #TODO:update hidden layer so self.layer_1 is used and its calculated optimally
            self.layer_1 *= 0
            self.layer_1 = sum(self.weights_0_1[review])
            
            output_layer_logits = np.dot(self.layer_1,self.weights_1_2)
            output_layer_predictions = self.sigmoid(output_layer_logits)
            #print("output layer",output_layer_predictions.shape)
            # TODO: Implement the back propagation pass here. (only update used weights)
            #       That means calculate the error for the forward pass's prediction
            #       and update the weights in the network according to their
            #       contributions toward the error, as calculated via the
            #       gradient descent and back propagation algorithms you 
            #       learned in class.
            error = output_layer_predictions - self.get_target_for_label(label)
            output_error_term = error*self.sigmoid_output_2_derivative(output_layer_predictions)
            hidden_error_term = self.weights_1_2*output_error_term
            
            delta_weights_h_o =  output_error_term * self.layer_1
            #TODO:Only update weights with input  = 1
            delta_weights_i_h = hidden_error_term#*layer0 #
            self.weights_1_2-= (self.learning_rate*delta_weights_h_o).reshape(self.weights_1_2.shape)
            self.weights_0_1[review]-= (self.learning_rate*delta_weights_i_h).T
            # Keep track of correct predictions.
            if(output_layer_predictions >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(output_layer_predictions < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given review.
        """
        # TODO: Run a forward pass through the network, like you did in the
        #       "train" function. That means use the given review to 
        #       update the input layer, then calculate values for the hidden layer,
        #       and finally calculate the output layer.
        #
        #       Note: The review passed into this function for prediction 
        #             might come from anywhere, so you should convert it 
        #             to lower case prior to using it.
        review = review.lower()
        review_indices = set()
        
        words = review.split(' ')
        for word in words:
            if word in self.word2index:
                review_indices.add(self.word2index[word])
            
            
        hidden_output = sum(self.weights_0_1[list(review_indices)])
        output_logits = np.dot(hidden_output,self.weights_1_2)
        output = self.sigmoid(output_logits)
        # TODO: The output layer should now contain a prediction. 
        #       Return `POSITIVE` for predictions greater-than-or-equal-to `0.5`, 
        #       and `NEGATIVE` otherwise.
        return "POSITIVE" if output >= 0.5 else "NEGATIVE"

Run the following cell to recreate the network and train it once again.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):1662. #Correct:1823 #Trained:2501 Training Accuracy:72.8%
Progress:20.8% Speed(reviews/sec):1627. #Correct:3812 #Trained:5001 Training Accuracy:76.2%
Progress:31.2% Speed(reviews/sec):1634. #Correct:5892 #Trained:7501 Training Accuracy:78.5%
Progress:41.6% Speed(reviews/sec):1638. #Correct:8010 #Trained:10001 Training Accuracy:80.0%
Progress:52.0% Speed(reviews/sec):1631. #Correct:10140 #Trained:12501 Training Accuracy:81.1%
Progress:62.4% Speed(reviews/sec):1634. #Correct:12270 #Trained:15001 Training Accuracy:81.7%
Progress:72.9% Speed(reviews/sec):1636. #Correct:14397 #Trained:17501 Training Accuracy:82.2%
Progress:83.3% Speed(reviews/sec):1635. #Correct:16567 #Trained:20001 Training Accuracy:82.8%
Progress:93.7% Speed(reviews/sec):1632. #Correct:18744 #Trained:22501 Training Accuracy:83.3%
Progress:99.9% Speed(reviews/sec):1631. #Correct:20074 #Trained:24001 Training

That should have trained much better than the earlier attempts. Run the following cell to test your model with 1000 predictions.

In [ ]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):1426. #Correct:842 #Tested:1000 Testing Accuracy:84.2%

# End of Project 5. 
# Further Noise Reduction<a id='lesson_6'></a>

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network_sparse_2.png">

In [ ]:
# words most frequently seen in a review with a "POSITIVE" label
pos_neg_ratios.most_common()

[('edie', 4.6913478822291435),
 ('paulie', 4.07753744390572),
 ('felix', 3.152736022363656),
 ('polanski', 2.8233610476132043),
 ('matthau', 2.80672172860924),
 ('victoria', 2.681021528714291),
 ('mildred', 2.6026896854443837),
 ('gandhi', 2.538973871058276),
 ('flawless', 2.451005098112319),
 ('superbly', 2.26002547857525),
 ('perfection', 2.159484249353372),
 ('astaire', 2.1400661634962708),
 ('captures', 2.038619547159581),
 ('voight', 2.030170492673053),
 ('wonderfully', 2.0218960560332353),
 ('powell', 1.978345424808467),
 ('brosnan', 1.9547990964725592),
 ('lily', 1.9203768470501485),
 ('bakshi', 1.9029851043382795),
 ('lincoln', 1.9014583864844796),
 ('refreshing', 1.8551812956655511),
 ('breathtaking', 1.8481124057791867),
 ('bourne', 1.8478489358790986),
 ('lemmon', 1.8458266904983307),
 ('delightful', 1.8002701588959635),
 ('flynn', 1.7996646487351682),
 ('andrews', 1.7764919970972666),
 ('homer', 1.7692866133759964),
 ('beautifully', 1.7626953362841438),
 ('soccer', 1.757857

In [ ]:
# words most frequently seen in a review with a "NEGATIVE" label
list(reversed(pos_neg_ratios.most_common()))[0:30]

[('boll', -4.835282406618394),
 ('uwe', -4.527846102553548),
 ('seagal', -3.606606956743819),
 ('unwatchable', -3.232428791272904),
 ('stinker', -3.1843768086123894),
 ('mst', -2.931339111817795),
 ('incoherent', -2.918210222585477),
 ('unfunny', -2.677582826320177),
 ('waste', -2.6057506568316997),
 ('blah', -2.5574420885289504),
 ('horrid', -2.4729780789227265),
 ('pointless', -2.443723518768643),
 ('atrocious', -2.4146593168481547),
 ('redeeming', -2.3576171050548607),
 ('prom', -2.35030978724235),
 ('drivel', -2.33663658094709),
 ('lousy', -2.2975727999267512),
 ('worst', -2.2771908066767406),
 ('laughable', -2.2547849053935245),
 ('awful', -2.2179631752505466),
 ('poorly', -2.2115829815780015),
 ('wasting', -2.1955788734265296),
 ('remotely', -2.1882648359647474),
 ('existent', -2.0714733720306593),
 ('boredom', -1.987774353154012),
 ('miserably', -1.9851235895077681),
 ('sucks', -1.9798005773223373),
 ('uninspired', -1.9760572894714397),
 ('lame', -1.9745380983043672),
 ('insult'

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
hist, edges = np.histogram(list(map(lambda x:x[1],pos_neg_ratios.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Word Positive/Negative Affinity Distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  """Entry point for launching an IPython kernel.


In [ ]:
frequency_frequency = Counter()

for word, cnt in total_counts.most_common():
    frequency_frequency[cnt] += 1

In [ ]:
hist, edges = np.histogram(list(map(lambda x:x[1],frequency_frequency.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="The frequency distribution of the words in our corpus")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  """Entry point for launching an IPython kernel.


# Project 6: Reducing Noise by Strategically Reducing the Vocabulary<a id='project_6'></a>

**TODO:** Improve `SentimentNetwork`'s performance by reducing more noise in the vocabulary. Specifically, do the following:
* Copy the `SentimentNetwork` class from the previous project into the following cell.
* Modify `pre_process_data`:
>* Add two additional parameters: `min_count` and `polarity_cutoff`
>* Calculate the positive-to-negative ratios of words used in the reviews. (You can use code you've written elsewhere in the notebook, but we are moving it into the class like we did with other helper code earlier.)
>* Andrew's solution only calculates a postive-to-negative ratio for words that occur at least 50 times. This keeps the network from attributing too much sentiment to rarer words. You can choose to add this to your solution if you would like.  
>* Change so words are only added to the vocabulary if they occur in the vocabulary more than `min_count` times.
>* Change so words are only added to the vocabulary if the absolute value of their postive-to-negative ratio is at least `polarity_cutoff`
* Modify `__init__`:
>* Add the same two parameters (`min_count` and `polarity_cutoff`) and use them when you call `pre_process_data`

In [ ]:
# TODO: -Copy the SentimentNetwork class from Project 5 lesson
#       -Modify it according to the above instructions 
# TODO: -Copy the SentimentNetwork class from Project 4 lesson
#       -Modify it according to the above instructions 
#       -Modify it to reduce noise, like in the video 
import time
import sys
import numpy as np

# Encapsulate our neural network in a class
class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 10, learning_rate = 0.1,min_count=50,polarity_cutoff=1):
        """Create a SentimenNetwork with the given settings
        Args:
            reviews(list) - List of reviews used for training
            labels(list) - List of POSITIVE/NEGATIVE labels associated with the given reviews
            hidden_nodes(int) - Number of nodes to create in the hidden layer
            learning_rate(float) - Learning rate to use while training
        
        """
        # Assign a seed to our random number generator to ensure we get
        # reproducable results during development 
        np.random.seed(1)

        # process the reviews and their associated labels so that everything
        # is ready for training
        self.pre_process_data(reviews, labels ,min_count,polarity_cutoff)
        
        # Build the network to have the number of hidden nodes and the learning rate that
        # were passed into this initializer. Make the same number of input nodes as
        # there are vocabulary words and create a single output node.
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels,min_count,polarity_cutoff):
        positive_counts = Counter()
        negative_counts = Counter()
        total_counts = Counter()
        pos_neg_ratios = Counter()
        review_vocab = set()
        # TODO: populate review_vocab with all of the words in the given reviews
        #       Remember to split reviews into individual words 
        #       using "split(' ')" instead of "split()".
        
        for index,label in enumerate(labels):
            words = reviews[index].split(' ')
            if label == 'POSITIVE':
                for word in words:
                    positive_counts[word]+=1
                    total_counts[word]+=1
                    #review_vocab.add(word)
            else:
                for word in words:
                    negative_counts[word]+=1
                    total_counts[word]+=1
                    
        for word in total_counts:
            count = total_counts[word]
            positive_count = positive_counts[word]
            negative_count = negative_counts[word] + 1
            word_positive_ratio  =  positive_count/negative_count
            word_positive_ratio = np.log(word_positive_ratio) if word_positive_ratio > 1 else  -np.log(1/(word_positive_ratio + 0.01))
            
            if (count > min_count and abs(word_positive_ratio)>= polarity_cutoff) or count > min_count:
                review_vocab.add(word)
        # Convert the vocabulary set to a list so we can access words via indices
        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        # TODO: populate label_vocab with all of the words in the given labels.
        #       There is no need to split the labels because each one is a single word.
        
        for label in labels:
            label_vocab.add(label)
        # Convert the label vocabulary set to a list so we can access labels via indices
        self.label_vocab = list(label_vocab)
        
        # Store the sizes of the review and label vocabularies.
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        # Create a dictionary of words in the vocabulary mapped to index positions
        self.word2index = {}
        # TODO: populate self.word2index with indices for all the words in self.review_vocab
        #       like you saw earlier in the notebook
        for index,word in enumerate(self.review_vocab):
            self.word2index[word] = index
        # Create a dictionary of labels mapped to index positions
        self.label2index = {}
        # TODO: do the same thing you did for self.word2index and self.review_vocab, 
        #       but for self.label2index and self.label_vocab instead
        for index,label in enumerate(self.label_vocab):
            self.label2index[label] = index
            
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Store the number of nodes in input, hidden, and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Store the learning rate
        self.learning_rate = learning_rate

        # Initialize weights
        
        # TODO: initialize self.weights_0_1 as a matrix of zeros. These are the weights between
        #       the input layer and the hidden layer.
        self.weights_0_1 = np.zeros((input_nodes,hidden_nodes))
        
        # TODO: initialize self.weights_1_2 as a matrix of random values. 
        #       These are the weights between the hidden layer and the output layer.
        self.weights_1_2 = np.random.normal(0.0, self.output_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        # TODO: Create the input layer, a two-dimensional matrix with shape 
        #       1 x input_nodes, with all values initialized to zero
        self.layer_1 = np.zeros((1,hidden_nodes))
    
                
    def get_target_for_label(self,label):
        # TODO: Copy the code you wrote for get_target_for_label 
        #       earlier in this notebook. 
        return 0 if label == 'NEGATIVE' else 1
        
    def sigmoid(self,x):
        # TODO: Return the result of calculating the sigmoid activation function
        #       shown in the lectures
        return 1 / ( 1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        # TODO: Return the derivative of the sigmoid activation function, 
        #       where "output" is the original output from the sigmoid fucntion 
        return output * (1 - output)

    def train(self, training_reviews_raw, training_labels):
        
        # make sure out we have a matching number of reviews and labels
        assert(len(training_reviews_raw) == len(training_labels))
        #create a list of lists, each inner list contains the word2index index for a review
        training_reviews = list()
        

        for review in training_reviews_raw:
            review_indices = set()
            words = review.split(' ')
            for word in words:
                if word in self.word2index:
                    review_indices.add(self.word2index[word])
            training_reviews.append(list(review_indices))
        # Keep track of correct predictions to display accuracy during training 
        correct_so_far = 0
        
        # Remember when we started for printing time statistics
        start = time.time()

        # loop through all the given reviews and run a forward and backward pass,
        # updating weights for every item
        for i in range(len(training_reviews)):
            
            # TODO: Get the next review and its correct label
            review = training_reviews[i]
            label = training_labels[i]
            # TODO: Implement the forward pass through the network. 
            #       That means use the given review to update the input layer, 
            #       then calculate values for the hidden layer,
            #       and finally calculate the output layer.
            # 
            #       Do not use an activation function for the hidden layer,
            #       but use the sigmoid activation function for the output layer.
            
            #TODO:update hidden layer so self.layer_1 is used and its calculated optimally
            #print("words:",review)
            self.layer_1 *= 0
            self.layer_1 = sum(self.weights_0_1[review])
            
            output_layer_logits = np.dot(self.layer_1,self.weights_1_2)
            output_layer_predictions = self.sigmoid(output_layer_logits)

            #print("output layer",output_layer_predictions.shape)
            # TODO: Implement the back propagation pass here. (only update used weights)
            #       That means calculate the error for the forward pass's prediction
            #       and update the weights in the network according to their
            #       contributions toward the error, as calculated via the
            #       gradient descent and back propagation algorithms you 
            #       learned in class.
            error = output_layer_predictions - self.get_target_for_label(label)
            output_error_term = error*self.sigmoid_output_2_derivative(output_layer_predictions)
            hidden_error_term = self.weights_1_2*output_error_term
            
            delta_weights_h_o =  output_error_term * self.layer_1
            #TODO:Only update weights with input  = 1
            delta_weights_i_h = hidden_error_term#*layer0 #
            self.weights_1_2-= (self.learning_rate*delta_weights_h_o).reshape(self.weights_1_2.shape)
            self.weights_0_1[review]-= (self.learning_rate*delta_weights_i_h).T
            # Keep track of correct predictions.
            if(output_layer_predictions >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(output_layer_predictions < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given review.
        """
        # TODO: Run a forward pass through the network, like you did in the
        #       "train" function. That means use the given review to 
        #       update the input layer, then calculate values for the hidden layer,
        #       and finally calculate the output layer.
        #
        #       Note: The review passed into this function for prediction 
        #             might come from anywhere, so you should convert it 
        #             to lower case prior to using it.
        review = review.lower()
        review_indices = set()
        
        words = review.split(' ')
        for word in words:
            if word in self.word2index:
                review_indices.add(self.word2index[word])
            
            
        hidden_output = sum(self.weights_0_1[list(review_indices)])
        output_logits = np.dot(hidden_output,self.weights_1_2)
        output = self.sigmoid(output_logits)
        # TODO: The output layer should now contain a prediction. 
        #       Return `POSITIVE` for predictions greater-than-or-equal-to `0.5`, 
        #       and `NEGATIVE` otherwise.
        return "POSITIVE" if output >= 0.5 else "NEGATIVE"

Run the following cell to train your network with a small polarity cutoff.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000],min_count=20,polarity_cutoff=0.05,learning_rate=0.01)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):1683. #Correct:1955 #Trained:2501 Training Accuracy:78.1%
Progress:20.8% Speed(reviews/sec):1644. #Correct:3997 #Trained:5001 Training Accuracy:79.9%
Progress:31.2% Speed(reviews/sec):1652. #Correct:6115 #Trained:7501 Training Accuracy:81.5%
Progress:41.6% Speed(reviews/sec):1659. #Correct:8267 #Trained:10001 Training Accuracy:82.6%
Progress:52.0% Speed(reviews/sec):1659. #Correct:10425 #Trained:12501 Training Accuracy:83.3%
Progress:62.4% Speed(reviews/sec):1661. #Correct:12562 #Trained:15001 Training Accuracy:83.7%
Progress:72.9% Speed(reviews/sec):1660. #Correct:14662 #Trained:17501 Training Accuracy:83.7%
Progress:83.3% Speed(reviews/sec):1662. #Correct:16827 #Trained:20001 Training Accuracy:84.1%
Progress:93.7% Speed(reviews/sec):1663. #Correct:19006 #Trained:22501 Training Accuracy:84.4%
Progress:99.9% Speed(reviews/sec):1663. #Correct:20328 #Trained:24001 Training

And run the following cell to test it's performance. It should be 

In [ ]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):1444. #Correct:859 #Tested:1000 Testing Accuracy:85.9%

Run the following cell to train your network with a much larger polarity cutoff.

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000],min_count=20,polarity_cutoff=0.8,learning_rate=0.01)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):1645. #Correct:1955 #Trained:2501 Training Accuracy:78.1%
Progress:20.8% Speed(reviews/sec):1645. #Correct:3997 #Trained:5001 Training Accuracy:79.9%
Progress:31.2% Speed(reviews/sec):1640. #Correct:6115 #Trained:7501 Training Accuracy:81.5%
Progress:41.6% Speed(reviews/sec):1641. #Correct:8267 #Trained:10001 Training Accuracy:82.6%
Progress:52.0% Speed(reviews/sec):1646. #Correct:10425 #Trained:12501 Training Accuracy:83.3%
Progress:62.4% Speed(reviews/sec):1650. #Correct:12562 #Trained:15001 Training Accuracy:83.7%
Progress:72.9% Speed(reviews/sec):1660. #Correct:14662 #Trained:17501 Training Accuracy:83.7%
Progress:83.3% Speed(reviews/sec):1660. #Correct:16827 #Trained:20001 Training Accuracy:84.1%
Progress:93.7% Speed(reviews/sec):1651. #Correct:19006 #Trained:22501 Training Accuracy:84.4%
Progress:99.9% Speed(reviews/sec):1651. #Correct:20328 #Trained:24001 Training

And run the following cell to test it's performance.

In [ ]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):1404. #Correct:859 #Tested:1000 Testing Accuracy:85.9%

In [ ]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000],min_count=200,polarity_cutoff=2.2,learning_rate=0.01)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):1769. #Correct:1912 #Trained:2501 Training Accuracy:76.4%
Progress:20.8% Speed(reviews/sec):1734. #Correct:3935 #Trained:5001 Training Accuracy:78.6%
Progress:31.2% Speed(reviews/sec):1746. #Correct:6018 #Trained:7501 Training Accuracy:80.2%
Progress:41.6% Speed(reviews/sec):1755. #Correct:8138 #Trained:10001 Training Accuracy:81.3%
Progress:52.0% Speed(reviews/sec):1751. #Correct:10271 #Trained:12501 Training Accuracy:82.1%
Progress:62.4% Speed(reviews/sec):1753. #Correct:12385 #Trained:15001 Training Accuracy:82.5%
Progress:72.9% Speed(reviews/sec):1746. #Correct:14471 #Trained:17501 Training Accuracy:82.6%
Progress:83.3% Speed(reviews/sec):1742. #Correct:16618 #Trained:20001 Training Accuracy:83.0%
Progress:93.7% Speed(reviews/sec):1742. #Correct:18777 #Trained:22501 Training Accuracy:83.4%
Progress:99.9% Speed(reviews/sec):1740. #Correct:20081 #Trained:24001 Training

In [ ]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:99.9% Speed(reviews/sec):1577. #Correct:847 #Tested:1000 Testing Accuracy:84.7%

# End of Project 6. 


# Analysis: What's Going on in the Weights?<a id='lesson_7'></a>

In [ ]:
mlp_full = SentimentNetwork(reviews[:-1000],labels[:-1000],min_count=0,polarity_cutoff=0,learning_rate=0.01)

In [ ]:
mlp_full.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):1643. #Correct:1962 #Trained:2501 Training Accuracy:78.4%
Progress:20.8% Speed(reviews/sec):1630. #Correct:4002 #Trained:5001 Training Accuracy:80.0%
Progress:31.2% Speed(reviews/sec):1615. #Correct:6120 #Trained:7501 Training Accuracy:81.5%
Progress:41.6% Speed(reviews/sec):1608. #Correct:8271 #Trained:10001 Training Accuracy:82.7%
Progress:52.0% Speed(reviews/sec):1598. #Correct:10431 #Trained:12501 Training Accuracy:83.4%
Progress:62.4% Speed(reviews/sec):1599. #Correct:12565 #Trained:15001 Training Accuracy:83.7%
Progress:72.9% Speed(reviews/sec):1602. #Correct:14670 #Trained:17501 Training Accuracy:83.8%
Progress:83.3% Speed(reviews/sec):1601. #Correct:16833 #Trained:20001 Training Accuracy:84.1%
Progress:93.7% Speed(reviews/sec):1604. #Correct:19015 #Trained:22501 Training Accuracy:84.5%
Progress:99.9% Speed(reviews/sec):1607. #Correct:20336 #Trained:24001 Training

<img src="https://raw.githubusercontent.com/llealgt/moview_review_sentiment_prediction_neural_network/master/sentiment_network_sparse.png">

In [ ]:
def get_most_similar_words(focus = "horrible"):
    most_similar = Counter()

    for word in mlp_full.word2index.keys():
        most_similar[word] = np.dot(mlp_full.weights_0_1[mlp_full.word2index[word]],mlp_full.weights_0_1[mlp_full.word2index[focus]])
    
    return most_similar.most_common()

In [ ]:
get_most_similar_words("excellent")

[('excellent', 0.13672950757352473),
 ('perfect', 0.12548286087225946),
 ('amazing', 0.09182763392599971),
 ('today', 0.09022366269441422),
 ('wonderful', 0.0893559769622146),
 ('fun', 0.0875044666742069),
 ('great', 0.08714175888229206),
 ('best', 0.08581088561788061),
 ('liked', 0.07769762912384341),
 ('definitely', 0.07667096405563839),
 ('brilliant', 0.07346604141795139),
 ('loved', 0.07328542892812215),
 ('favorite', 0.07278113603616077),
 ('superb', 0.07173620717850507),
 ('fantastic', 0.07092219191626623),
 ('job', 0.0692027998563064),
 ('incredible', 0.0664240779526144),
 ('enjoyable', 0.0656325605028888),
 ('rare', 0.06481921266261509),
 ('highly', 0.06388945335097052),
 ('enjoyed', 0.06212754610181297),
 ('wonderfully', 0.06205517860409017),
 ('perfectly', 0.061093208811887394),
 ('fascinating', 0.06066354793749387),
 ('bit', 0.05969760969432539),
 ('gem', 0.05951085929615679),
 ('outstanding', 0.05886080814708301),
 ('beautiful', 0.058613934703162056),
 ('surprised', 0.05831

In [ ]:
get_most_similar_words("terrible")

[('worst', 0.16966107259049845),
 ('awful', 0.12026847019691246),
 ('waste', 0.11945367265311006),
 ('poor', 0.09275888757443551),
 ('terrible', 0.09142538719772796),
 ('dull', 0.08420927167822362),
 ('poorly', 0.08124154451604207),
 ('disappointment', 0.08006475962136872),
 ('fails', 0.07859977372333754),
 ('disappointing', 0.07733948548032338),
 ('boring', 0.07709336533844613),
 ('unfortunately', 0.07550244970585908),
 ('worse', 0.07060183536419468),
 ('mess', 0.07056429962359041),
 ('stupid', 0.06948482283254305),
 ('badly', 0.06688890366622859),
 ('annoying', 0.06568702190337415),
 ('bad', 0.06309381453757217),
 ('save', 0.06288059749586575),
 ('disappointed', 0.06269235381207287),
 ('wasted', 0.061387183028051295),
 ('supposed', 0.06098545295772518),
 ('horrible', 0.06012177233938014),
 ('laughable', 0.05869840628546763),
 ('crap', 0.05810452866788459),
 ('basically', 0.05721884036963617),
 ('nothing', 0.0571582200430342),
 ('ridiculous', 0.056905481068931445),
 ('lacks', 0.055766

In [ ]:
import matplotlib.colors as colors

words_to_visualize = list()
for word, ratio in pos_neg_ratios.most_common(500):
    if(word in mlp_full.word2index.keys()):
        words_to_visualize.append(word)
    
for word, ratio in list(reversed(pos_neg_ratios.most_common()))[0:500]:
    if(word in mlp_full.word2index.keys()):
        words_to_visualize.append(word)

In [ ]:
pos = 0
neg = 0

colors_list = list()
vectors_list = list()
for word in words_to_visualize:
    if word in pos_neg_ratios.keys():
        vectors_list.append(mlp_full.weights_0_1[mlp_full.word2index[word]])
        if(pos_neg_ratios[word] > 0):
            pos+=1
            colors_list.append("#00ff00")
        else:
            neg+=1
            colors_list.append("#000000")

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(vectors_list)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="vector T-SNE for most polarized words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_to_visualize))

p.scatter(x="x1", y="x2", size=8, source=source,color=colors_list)

word_labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(word_labels)

show(p)

# green indicates positive words, black indicates negative words

RuntimeError: ignored